# SKU Discount Handler Module

This module handles the complete lifecycle of SKU discounts based on Module 3 recommendations.

**Note:** This module is called from Module 3. Market data should already be refreshed in Module 3 before calling this handler.

## Workflow
1. **Deactivate Active SKU Discounts** - Clear existing discounts before creating new ones
2. **Select Target Retailers** - Identify which retailers should receive the discount
3. **Calculate Discount** - Determine discount percentage based on market/margin data
4. **Structure DataFrame** - Prepare data in API-expected format
5. **Push SKU Discount** - Upload to MaxAB API

## Input Requirements

**From Module 3 (with market data already refreshed):**
- `product_id`, `warehouse_id`, `sku`, `cohort_id`
- `activate_sku_discount` - Boolean flag
- `current_price`, `wac_p`

*Market Prices (as margins):*
- `below_market`, `market_min`, `market_25`, `market_50`, `market_75`, `market_max`, `above_market`

*Margin Tiers:*
- `margin_tier_below`, `margin_tier_1`, `margin_tier_2`, `margin_tier_3`, `margin_tier_4`, `margin_tier_5`, `margin_tier_above_1`, `margin_tier_above_2`

## Output
- Deactivated existing SKU discounts
- Created new SKU discounts for qualifying SKUs
- Summary report

## Usage (called from Module 3)
```python
%run sku_discount_handler.ipynb
result = process_sku_discounts(df_output, mode='testing')
```


In [ ]:
# =============================================================================
# IMPORTS & CONFIGURATION
# =============================================================================
import pandas as pd
import numpy as np
import json
import time
import base64
import os
from datetime import datetime, timedelta
import pytz

# AWS for secrets management
import boto3
from botocore.exceptions import ClientError

# HTTP requests for API calls
import requests

# Progress bar
from tqdm import tqdm

# Import setup_environment_2 for credentials
import sys
sys.path.append('..')
import setup_environment_2

# Initialize environment (loads Snowflake credentials)
setup_environment_2.initialize_env()

# Note: Market data is passed from Module 3 (already refreshed)
# No need to import market_data_module here

# Cairo Timezone
CAIRO_TZ = pytz.timezone('Africa/Cairo')
CAIRO_NOW = datetime.now(CAIRO_TZ)
TODAY = CAIRO_NOW.date()

# =============================================================================
# CONFIGURATION
# =============================================================================
# Default discount settings
DEFAULT_DISCOUNT_DURATION_HOURS = 12  # Discount valid until next run

# Market price columns (from market_data_module)
MARKET_PRICE_COLS = [
    'below_market', 'market_min', 'market_25', 'market_50', 
    'market_75', 'market_max', 'above_market'
]

# Margin tier columns (from market_data_module)
MARGIN_TIER_COLS = [
    'margin_tier_below', 'margin_tier_1', 'margin_tier_2', 'margin_tier_3', 
    'margin_tier_4', 'margin_tier_5', 'margin_tier_above_1', 'margin_tier_above_2'
]

print(f"SKU Discount Handler loaded at {CAIRO_NOW.strftime('%Y-%m-%d %H:%M:%S')} Cairo time")


In [ ]:
# =============================================================================
# AWS & API FUNCTIONS
# =============================================================================

def get_secret(secret_name: str) -> str:
    """
    Retrieve a secret from AWS Secrets Manager.
    
    Args:
        secret_name: The name/path of the secret in AWS Secrets Manager
    
    Returns:
        Secret string (JSON format) or decoded binary
    """
    region_name = "us-east-1"
    session = boto3.session.Session()
    client = session.client(service_name='secretsmanager', region_name=region_name)

    try:
        response = client.get_secret_value(SecretId=secret_name)
    except ClientError as e:
        print(f"AWS Error: {e}")
        raise e
    
    if 'SecretString' in response:
        return response['SecretString']
    return base64.b64decode(response['SecretBinary'])


def get_access_token(url: str, client_id: str, client_secret: str) -> str:
    """
    Get OAuth2 access token for MaxAB API authentication.
    """
    response = requests.post(
        url,
        data={
            "grant_type": "password",
            "username": API_USERNAME,
            "password": API_PASSWORD
        },
        auth=(client_id, client_secret),
    )
    return response.json()["access_token"]


def _get_api_token() -> str:
    """
    Get a fresh API token for MaxAB API requests.
    """
    return get_access_token(
        'https://sso.maxab.info/auth/realms/maxab/protocol/openid-connect/token',
        'main-system-externals',
        API_SECRET
    )

print("AWS & API functions defined ✓")


In [ ]:
# =============================================================================
# API CREDENTIALS INITIALIZATION
# =============================================================================
# Load API credentials from AWS Secrets Manager

pricing_api_secret = json.loads(get_secret("prod/pricing/api/"))
API_USERNAME = pricing_api_secret["egypt_username"]
API_PASSWORD = pricing_api_secret["egypt_password"]
API_SECRET = pricing_api_secret["egypt_secret"]

print("✓ API credentials loaded successfully")


In [ ]:
# =============================================================================
# FUNCTION 1: DEACTIVATE ACTIVE SKU DISCOUNTS
# =============================================================================
# Reference: Mustafa/Deactivate_HH.ipynb

import snowflake.connector
import math

def get_snowflake_timezone() -> str:
    """Get the Snowflake server timezone."""
    con = snowflake.connector.connect(
        user     = os.environ["SNOWFLAKE_USERNAME"],
        account  = os.environ["SNOWFLAKE_ACCOUNT"],
        password = os.environ["SNOWFLAKE_PASSWORD"],
        database = os.environ["SNOWFLAKE_DATABASE"]
    )
    try:
        cur = con.cursor()
        cur.execute("USE WAREHOUSE COMPUTE_WH")
        cur.execute("SHOW PARAMETERS LIKE 'TIMEZONE'")
        result = cur.fetchone()
        return result[1] if result else 'UTC'
    finally:
        cur.close()
        con.close()

# Get Snowflake timezone for query conversion
TIMEZONE = get_snowflake_timezone()
print(f"Snowflake timezone: {TIMEZONE}")

def query_snowflake(query: str) -> pd.DataFrame:
    """
    Execute a query against Snowflake and return results as DataFrame.
    """
    con = snowflake.connector.connect(
        user     = os.environ["SNOWFLAKE_USERNAME"],
        account  = os.environ["SNOWFLAKE_ACCOUNT"],
        password = os.environ["SNOWFLAKE_PASSWORD"],
        database = os.environ["SNOWFLAKE_DATABASE"]
    )

    try:
        cur = con.cursor()
        cur.execute("USE WAREHOUSE COMPUTE_WH")
        cur.execute(query)
        
        column_names = [col[0] for col in cur.description]
        results = cur.fetchall()
        
        if not results:
            out = pd.DataFrame(columns=[name.lower() for name in column_names])
        else:
            out = pd.DataFrame(np.array(results), columns=column_names)
            out.columns = out.columns.str.lower()
        
        return out
        
    except Exception as e:
        print(f"❌ Query error: {e}")
        raise
        
    finally:
        cur.close()
        con.close()


def _call_deactivate_api(ids: list) -> requests.Response:
    """
    Call MaxAB API to deactivate SKU discounts by their IDs.
    
    Args:
        ids: List of SKU discount IDs to deactivate
        
    Returns:
        API response object
    """
    token = _get_api_token()
    
    url = "https://api.maxab.info/commerce/api/admins/v1/sku-discounts/status"
    
    headers = {
        'Authorization': f'bearer {token}',
        'Content-Type': 'application/json'
    }
    
    payload = {
        "ids": ids,
        "isActivate": False
    }
    
    response = requests.put(url, headers=headers, json=payload)
    return response


def get_active_sku_discount_ids() -> list:
    """
    Query Snowflake to get all currently active SKU discount IDs.
    
    Returns:
        List of active SKU discount IDs
    """
    query = f'''
    SELECT sd.id AS sku_discount_id
    FROM SKU_DISCOUNTS sd
    WHERE active = 'true'
    AND CONVERT_TIMEZONE('{TIMEZONE}', 'Africa/Cairo', CURRENT_TIMESTAMP()) BETWEEN start_at AND end_at
    AND name_en = 'Special Discounts'
    '''
    
    df = query_snowflake(query)
    return df['sku_discount_id'].unique().tolist()


def deactivate_active_sku_discounts(mode: str = 'testing') -> dict:
    """
    Deactivate all currently active SKU discounts (name = 'Special Discounts').
    
    This function should be called FIRST before creating new discounts
    to ensure a clean slate (no duplicate/overlapping discounts).
    
    Process:
    1. Query Snowflake to get all active SKU discount IDs
    2. Break IDs into chunks of 10
    3. Call API for each chunk to deactivate
    
    Args:
        mode: 'testing' (don't call API) or 'live' (call API)
    
    Returns:
        dict with:
            - success: bool
            - deactivated_count: int
            - failed_count: int
            - errors: list
    """
    print(f"{'🧪' if mode == 'testing' else '🚀'} Deactivating SKU discounts (mode: {mode})")
    
    result = {
        "success": True,
        "deactivated_count": 0,
        "failed_count": 0,
        "errors": []
    }
    
    # Step 1: Get active SKU discount IDs from Snowflake
    print("  Querying active SKU discounts from Snowflake...")
    ids_list = get_active_sku_discount_ids()
    print(f"  Found {len(ids_list)} active SKU discounts to deactivate")
    
    if len(ids_list) == 0:
        print("  No active SKU discounts to deactivate")
        return result
    
    if mode == 'testing':
        print(f"  🧪 [TESTING] Would deactivate {len(ids_list)} SKU discounts (skipped)")
        result['deactivated_count'] = len(ids_list)
        return result
    
    # Step 2: Deactivate in chunks of 10
    chunk_size = 10
    total_chunks = math.ceil(len(ids_list) / chunk_size)
    failed_chunks = []
    
    print(f"  Deactivating in {total_chunks} chunks...")
    
    for i in tqdm(range(0, len(ids_list), chunk_size), total=total_chunks, desc="Deactivating SKU Discounts"):
        chunk = ids_list[i:i + chunk_size]
        
        response = _call_deactivate_api(chunk)
        
        if response.status_code == 200:
            result['deactivated_count'] += len(chunk)
        else:
            result['failed_count'] += len(chunk)
            failed_chunks.append({"ids": chunk, "error": response.text})
    
    # Step 3: Report results
    print(f"\n  ✓ Completed! Deactivated: {result['deactivated_count']}, Failed: {result['failed_count']}")
    
    if failed_chunks:
        result['success'] = False
        result['errors'] = failed_chunks
        print(f"  ✗ Failed chunks: {len(failed_chunks)}")
        for fail in failed_chunks[:5]:  # Show first 5 failures
            print(f"    - IDs: {fail['ids']}, Error: {fail['error'][:100]}...")
    
    return result


print("Function 1: deactivate_active_sku_discounts() defined ✓")


In [ ]:
# =============================================================================
# FUNCTION 2: SELECT TARGET RETAILERS
# =============================================================================

def select_target_retailers(
    df_skus: pd.DataFrame,
    selection_criteria: str = 'all'
) -> pd.DataFrame:
    """
    Select which retailers should receive the SKU discount.
    
    Different strategies for targeting:
    - 'all': All retailers in the warehouse/cohort
    - 'active': Only retailers who have purchased recently
    - 'dormant': Only retailers who haven't purchased recently
    - 'custom': Based on custom retailer list
    
    Args:
        df_skus: DataFrame with SKUs to discount
        selection_criteria: Targeting strategy
    
    Returns:
        DataFrame with retailer_ids column added
    """
    # TODO: Implement retailer selection logic
    # The user will provide the specific business logic
    
    print(f"Selecting target retailers with criteria: {selection_criteria}")
    
    # PLACEHOLDER: Add retailer targeting logic
    # This might involve:
    # - Querying active retailers from database
    # - Filtering by purchase history
    # - Applying cohort-specific rules
    
    df_result = df_skus.copy()
    df_result['retailer_ids'] = None  # Will be populated with list of retailer IDs
    
    return df_result


print("Function 2: select_target_retailers() defined ✓")


In [ ]:
# =============================================================================
# FUNCTION 3: CALCULATE DISCOUNT
# =============================================================================

def calculate_discount(
    row: pd.Series,
    market_prices: dict,
    margin_tiers: dict
) -> float:
    """
    Calculate the appropriate discount percentage for a SKU.
    
    Uses market prices and margin tiers to determine optimal discount
    that will stimulate demand while maintaining profitability.
    
    Args:
        row: Series containing SKU data (current_price, wac_p, etc.)
        market_prices: dict with keys:
            - below_market, market_min, market_25, market_50, 
            - market_75, market_max, above_market
        margin_tiers: dict with keys:
            - margin_tier_below, margin_tier_1, margin_tier_2, margin_tier_3,
            - margin_tier_4, margin_tier_5, margin_tier_above_1, margin_tier_above_2
    
    Returns:
        Discount percentage (e.g., 5.0 for 5%)
    """
    # TODO: Implement discount calculation logic
    # The user will provide the specific business logic
    
    current_price = row.get('current_price', 0)
    wac = row.get('wac_p', 0)
    
    # PLACEHOLDER: Calculate discount based on market position and margin
    # Example logic:
    # - If price is above market_75, larger discount to become competitive
    # - If price is below market_25, smaller discount to protect margin
    # - Consider margin tiers to ensure profitability
    
    discount = 0.0  # Default no discount
    
    return discount


def calculate_discounts_batch(df_skus: pd.DataFrame) -> pd.DataFrame:
    """
    Calculate discounts for all SKUs in batch.
    
    Args:
        df_skus: DataFrame with SKUs needing discounts
                 Must include market_prices and margin_tier columns
    
    Returns:
        DataFrame with 'discount_percentage' column added
    """
    print(f"Calculating discounts for {len(df_skus)} SKUs...")
    
    df_result = df_skus.copy()
    
    # Apply discount calculation to each row
    df_result['discount_percentage'] = df_result.apply(
        lambda row: calculate_discount(
            row,
            {col: row.get(col) for col in MARKET_PRICE_COLS},
            {col: row.get(col) for col in MARGIN_TIER_COLS}
        ),
        axis=1
    )
    
    return df_result


print("Function 3: calculate_discount() & calculate_discounts_batch() defined ✓")


In [ ]:
# =============================================================================
# FUNCTION 4: STRUCTURE DATAFRAME
# =============================================================================

def structure_sku_discount_dataframe(df_skus: pd.DataFrame) -> pd.DataFrame:
    """
    Structure the DataFrame in the format expected by the SKU discount API.
    
    Transforms the internal data format into the API-expected format
    with proper column names, data types, and required fields.
    
    Args:
        df_skus: DataFrame with discount calculations and retailer targeting
    
    Returns:
        DataFrame ready for API upload with columns:
            - product_id
            - warehouse_id
            - discount_percentage
            - retailer_ids (list or comma-separated)
            - start_datetime
            - end_datetime
            - (other API-required fields)
    """
    # TODO: Implement dataframe structuring
    # The user will provide the exact API format requirements
    
    print(f"Structuring {len(df_skus)} SKU discounts for API...")
    
    df_api = df_skus.copy()
    
    # PLACEHOLDER: Transform columns to API format
    # Example transformations:
    # - Rename columns to API naming convention
    # - Format datetime fields
    # - Convert retailer_ids to required format
    # - Add any required default values
    
    # Add start and end times
    df_api['start_datetime'] = CAIRO_NOW
    df_api['end_datetime'] = CAIRO_NOW + timedelta(hours=DEFAULT_DISCOUNT_DURATION_HOURS)
    
    return df_api


print("Function 4: structure_sku_discount_dataframe() defined ✓")


In [ ]:
# =============================================================================
# FUNCTION 5: PUSH SKU DISCOUNT
# =============================================================================

def push_sku_discount(df_api: pd.DataFrame, mode: str = 'testing') -> dict:
    """
    Push SKU discounts to MaxAB API.
    
    Args:
        df_api: DataFrame structured for API (from structure_sku_discount_dataframe)
        mode: 'testing' (don't upload) or 'live' (upload to API)
    
    Returns:
        dict with:
            - success: bool
            - created_count: int
            - failed_count: int
            - errors: list
    """
    # TODO: Implement API upload
    # The user will provide the exact API endpoint and format
    
    print(f"\n{'🧪' if mode == 'testing' else '🚀'} MODE: {mode.upper()}")
    
    result = {
        "success": True,
        "created_count": 0,
        "failed_count": 0,
        "errors": []
    }
    
    if df_api.empty:
        print("No SKU discounts to push")
        return result
    
    print(f"Pushing {len(df_api)} SKU discounts...")
    
    if mode == 'testing':
        print("  🧪 [TESTING] Would push SKU discounts (skipped)")
        result['created_count'] = len(df_api)
        return result
    
    # PLACEHOLDER: Replace with actual API call
    # token = _get_api_token()
    # for idx, row in tqdm(df_api.iterrows(), total=len(df_api)):
    #     response = requests.post(
    #         "https://api.maxab.info/...",
    #         headers={"Authorization": f"bearer {token}"},
    #         json={...}
    #     )
    #     if response.ok:
    #         result['created_count'] += 1
    #     else:
    #         result['failed_count'] += 1
    #         result['errors'].append(...)
    
    result['success'] = result['failed_count'] == 0
    return result


print("Function 5: push_sku_discount() defined ✓")


In [ ]:
# =============================================================================
# MAIN ORCHESTRATION FUNCTION
# =============================================================================

def process_sku_discounts(
    df_module3_output: pd.DataFrame,
    mode: str = 'testing'
) -> dict:
    """
    Main entry point: Process SKU discounts from Module 3 output.
    
    Note: Market data should already be refreshed in Module 3 before calling this.
    
    Orchestrates the full workflow:
    1. Deactivate existing SKU discounts
    2. Filter to SKUs needing discounts (activate_sku_discount = True)
    3. Select target retailers
    4. Calculate discount percentages
    5. Structure DataFrame for API
    6. Push to API
    
    Args:
        df_module3_output: DataFrame from Module 3 with columns:
            - product_id, warehouse_id, sku, cohort_id
            - activate_sku_discount (bool)
            - current_price, wac_p
            - Market price columns (already refreshed in Module 3)
            - Margin tier columns (already refreshed in Module 3)
        mode: 'testing' or 'live'
    
    Returns:
        dict with summary of all operations
    """
    print("\n" + "="*70)
    print("SKU DISCOUNT HANDLER")
    print("="*70)
    print(f"Mode: {mode}")
    print(f"Input records: {len(df_module3_output)}")
    
    result = {
        'timestamp': CAIRO_NOW.strftime('%Y-%m-%d %H:%M:%S'),
        'mode': mode,
        'total_input': len(df_module3_output),
        'to_activate': 0,
        'deactivated': 0,
        'created': 0,
        'failed': 0
    }
    
    df = df_module3_output.copy()
    
    # Step 1: Deactivate existing discounts
    print("\n" + "-"*50)
    print("STEP 1: Deactivating existing SKU discounts")
    print("-"*50)
    deactivate_result = deactivate_active_sku_discounts(mode=mode)
    result['deactivated'] = deactivate_result['deactivated_count']
    
    # Step 2: Filter to SKUs needing discounts
    print("\n" + "-"*50)
    print("STEP 2: Filtering SKUs for discount")
    print("-"*50)
    df_to_activate = df[
        df['activate_sku_discount'] == True
    ].copy()
    result['to_activate'] = len(df_to_activate)
    print(f"SKUs to activate: {len(df_to_activate)}")
    
    if df_to_activate.empty:
        print("No SKUs need discounts. Done.")
        return result
    
    # Step 3: Select target retailers
    print("\n" + "-"*50)
    print("STEP 3: Selecting target retailers")
    print("-"*50)
    df_with_retailers = select_target_retailers(df_to_activate)
    
    # Step 4: Calculate discounts
    print("\n" + "-"*50)
    print("STEP 4: Calculating discount percentages")
    print("-"*50)
    df_with_discounts = calculate_discounts_batch(df_with_retailers)
    
    # Step 5: Structure DataFrame
    print("\n" + "-"*50)
    print("STEP 5: Structuring data for API")
    print("-"*50)
    df_api = structure_sku_discount_dataframe(df_with_discounts)
    
    # Step 6: Push to API
    print("\n" + "-"*50)
    print("STEP 6: Pushing to API")
    print("-"*50)
    push_result = push_sku_discount(df_api, mode=mode)
    result['created'] = push_result['created_count']
    result['failed'] = push_result['failed_count']
    
    # Summary
    print("\n" + "="*70)
    print("SUMMARY")
    print("="*70)
    print(f"Mode: {result['mode']}")
    print(f"Total input: {result['total_input']}")
    print(f"Discounts deactivated: {result['deactivated']}")
    print(f"SKUs to activate: {result['to_activate']}")
    print(f"Discounts created: {result['created']}")
    print(f"Discounts failed: {result['failed']}")
    
    return result


print("Main function: process_sku_discounts() defined ✓")


In [ ]:
# =============================================================================
# USAGE EXAMPLE (Called from Module 3)
# =============================================================================
# 
# This module is designed to be called from Module 3 after the periodic
# action engine has determined which SKUs need discounts.
#
# Example usage:
#
# %run sku_discount_handler.ipynb
#
# # df_output already contains market prices and margin tiers from data extraction
# # Required columns:
# #   - product_id, warehouse_id, sku, cohort_id
# #   - activate_sku_discount (bool)
# #   - current_price, wac_p
# #   - below_market, market_min, market_25, market_50, market_75, market_max, above_market
# #   - margin_tier_below, margin_tier_1, ..., margin_tier_above_2
#
# result = process_sku_discounts(df_output, mode='testing')
#
# print(f"Created {result['created']} SKU discounts")

print("\n" + "="*60)
print("SKU DISCOUNT HANDLER MODULE READY")
print("="*60)
print("\nRequired input columns from Module 3:")
print("  - product_id, warehouse_id, sku, cohort_id")
print("  - activate_sku_discount (bool)")
print("  - current_price, wac_p")
print("\nRequired market price columns:")
for col in MARKET_PRICE_COLS:
    print(f"  - {col}")
print("\nRequired margin tier columns:")
for col in MARGIN_TIER_COLS:
    print(f"  - {col}")
print("\nUsage: result = process_sku_discounts(df_output, mode='testing')")
